In [1]:
import tensorflow as tf
import numpy as np
from sklearn.neighbors import NearestNeighbors
import Nearest_Neighbor as nn
import random
from joblib import Parallel, delayed

import importlib
importlib.reload(nn)

<module 'Nearest_Neighbor' from 'D:\\UCSD\\F17\\CSE293\\wordtranslation\\model\\Nearest_Neighbor.py'>

In [2]:
train_data_file = "D:/UCSD/F17/CSE293/data_prep_scripts/train_data.npy"
train_data = np.load(train_data_file)
test_data_file = "D:/UCSD/F17/CSE293/data_prep_scripts/test_data.npy"
test_data = np.load(test_data_file)


#train_data_normalized_file = "D:/UCSD/F17/CSE293/data_prep_scripts/train_data_normalized.npy"
#train_data_normalized = np.load(train_data_normalized_file)

#test_data_normalized_file = "D:/UCSD/F17/CSE293/data_prep_scripts/test_data_normalized.npy"
#test_data_normalized = np.load(test_data_normalized_file)

#row-wise normalizing make sure that the data is arranged rowwise
####### ran once to preprocess.

def normalize(X):
    X = (X / np.linalg.norm(X, axis=1)[:,None])
    return X

for lang_data in test_data: #each language
    for item_e in enumerate(lang_data): #for each example
        i, item = item_e
        lang_data[i] = normalize(item) #[[source], [target]]
        
np.save('D:/UCSD/F17/CSE293/data_prep_scripts/test_data_normalized.npy', test_data)

In [3]:
nearest_ne = nn.Nearest_Neighbor(embeddings_dir="D:/UCSD/F17/CSE293/", domain_dir='D:/UCSD/F17/CSE293/wordtranslation/model/data/', lang_list=['es', 'pt'], k_neighbors=10, flag='test')
             
nearest_ne_train = dict()
'''lang_set = set()
for lang_pairs in train_lang_pairs:
    langs = llang_set = set()
for lang_pairs in train_lang_pairs:
    langs = lang_pairs.split('-')
    lang_set.update(langs)
ang_pairs.split('-')
    lang_set.update(langs)
'''

nearest_ne_train = nn.Nearest_Neighbor(embeddings_dir="D:/UCSD/F17/CSE293", domain_dir='D:/UCSD/F17/CSE293/wordtranslation/model/data/', lang_list=['pt','es','en'], k_neighbors=80, flag='train')


In [4]:
def dist(point, cur_point):
    return np.linalg.norm(point-cur_point)

def farthest_points(row, cur_point):
    dist_list = np.apply_along_axis(dist, 1, row, cur_point)
    indices = np.argsort(dist_list)
    return row[indices[80//2:]]

def match(nn_list, y):
    if y in nn_list:
        return 1
    else:
        return 0

In [8]:
#increase gamma makes model more strict on distance from negative expample i.e. farther from negative, closer to actual
# k is number of negative examples to be passed to calculate loss
class model(object):
    def __init__(self, ckpt_path, lr, dim_lang=5, dim_model=300, k=40, gamma=0.6, model_name="MLPM"):
        #self.epochs = epochs
        self.ckpt_path = ckpt_path
        self.model_name = model_name
        self.lr = lr
        self.dim_lang = dim_lang
        self.dim_model = dim_model
        self.all_lang_rep_enc = dict()
        self.all_lang_rep_dec = dict()
        self.k = k
        for lang in ["en", "pt", "es"]:
            self.all_lang_rep_enc[lang] = np.random.normal(loc=0.0, scale=1/np.sqrt(float(dim_lang)), size=(dim_lang))
            self.all_lang_rep_dec[lang] = np.random.normal(loc=0.0, scale=1/np.sqrt(float(dim_lang)), size=(dim_lang))
        
        def __graph__():
            tf.reset_default_graph()
            self.graph = tf.Graph()
            with self.graph.as_default():

                # source and target vector representation
                self.source_words = tf.placeholder(tf.float32, shape=[None, dim_model])
                self.target_words = tf.placeholder(tf.float32, shape=[None, dim_model])

                self.max_margin_neighbors = tf.placeholder(tf.float32, shape=[None, self.k, dim_model])
                # parameter matrices
                self.encoder = tf.Variable(tf.truncated_normal([dim_model, dim_model],\
                                           stddev=1/tf.sqrt(float(dim_model))), name='encoder')

                self.decoder = tf.Variable(tf.truncated_normal([dim_model, dim_model],\
                                           stddev=1/tf.sqrt(float(dim_model))), name='decoder')

                #todo:: second dimension can be different that dim_model
                self.lang_encoder = tf.Variable(tf.truncated_normal([dim_lang,dim_model],\
                                               stddev=1/tf.sqrt(float(dim_model))), name='lang_encoder')
                self.lang_decoder = tf.Variable(tf.truncated_normal([dim_lang,dim_model],\
                                               stddev=1/tf.sqrt(float(dim_model))), name='lang_decoder')

                # language representations
                self.lang_rep_enc_placeholder = tf.placeholder(tf.float32, shape=[1,self.dim_lang])
                self.lang_rep_enc = tf.Variable(tf.truncated_normal([1, self.dim_lang],\
                                           stddev=1/tf.sqrt(1.0*dim_lang)))
                self.assign_lang_rep_enc_op = self.lang_rep_enc.assign(self.lang_rep_enc_placeholder)
                
                self.lang_rep_dec_placeholder = tf.placeholder(tf.float32, shape=[1,self.dim_lang])
                self.lang_rep_dec = tf.Variable(tf.truncated_normal([1, self.dim_lang],\
                                           stddev=1/tf.sqrt(1.0*dim_lang)))
                self.assign_lang_rep_dec_op = self.lang_rep_dec.assign(self.lang_rep_dec_placeholder)

                # model equation
                self.target_pred = self.get_model(self.encoder, self.decoder, self.source_words,\
                                                  self.lang_rep_enc, self.lang_rep_dec, self.lang_encoder, self.lang_decoder)

                # cosine distance between target_words and predicted target words
                # A: None*1
                A = tf.losses.cosine_distance(tf.nn.l2_normalize(self.target_words,dim=1),\
                                              tf.nn.l2_normalize(self.target_pred,dim=1),\
                                              dim=1, reduction="none")
                # repeat A k times to use it later for max-margin
                # B: None*k
                B = tf.tile(A, [1, self.k])

                # repeat target_pred k times on row
                # target_pred_k: None*k*dim_model
                
                #todo:: make target pred normalized before repeat
                #target_pred_normalized = tf.nn.l2_normalize(self.target_pred,dim=1)
                target_pred_k = tf.tile(self.target_pred,[1, self.k])
                target_pred_k = tf.reshape(target_pred_k, [tf.shape(self.target_pred)[0], self.k, dim_model])

                target_pred_k_normalize = tf.nn.l2_normalize(target_pred_k, dim=2)

                max_margin_neighbors_normalize = tf.nn.l2_normalize(self.max_margin_neighbors, dim=2)


                # cosine distance between target_pred_normalize and k negative targets
                # C: None*k*1 and reshape to None*k
                C = tf.losses.cosine_distance(target_pred_k_normalize, max_margin_neighbors_normalize, dim=2)
                C = tf.reshape(C, tf.shape(C)[:2])

                #max-margin loss
                self.loss = tf.reduce_sum(tf.maximum(0.,B-C+gamma))
                #squared loss
                #self.loss = tf.reduce_sum(tf.square(self.target_words-self.target_pred)

                #self.train_step = tf.train.GradientDescentOptimizer(self.lr).minimize(self.loss)
                self.train_step = tf.train.AdamOptimizer(learning_rate=self.lr).minimize(self.loss)
                self.init = tf.global_variables_initializer()
                self.saver = tf.train.Saver()
        
        print('start building graph')
        __graph__()
        print('graph built')
        
    # encoder, decoder: 300x300   source_words: Nonex300    lang_rep: 1xdim_lang    lang_coder: dim_langx300
    # Output: Nonex300
    # we calculate the final projection in two parts  
    def get_model(self, encoder, decoder, source_words, lang_rep_enc, lang_rep_dec, lang_encoder, lang_decoder):
        #source to shared using encoder
        shared_source_words = tf.matmul(source_words, encoder)
        shared_lang_rep_enc = tf.matmul(lang_rep_enc, lang_encoder)
        num_examples = tf.shape(shared_source_words)[0]
        #(shared_source_words.get_shape())[0]
        tile = tf.tile(shared_lang_rep_enc, tf.convert_to_tensor([1, num_examples]))
        shared_lang_aux = tf.reshape(tile, [num_examples, self.dim_model])
        shared_embedding_vector = shared_source_words + shared_lang_aux
        
        #shared to target using decoder
        decoded_word_base = tf.matmul(shared_embedding_vector, decoder)
        shared_lang_rep_dec = tf.matmul(lang_rep_dec, lang_decoder)
        tile = tf.tile(shared_lang_rep_dec, tf.convert_to_tensor([1, num_examples]))
        shared_lang_aux = tf.reshape(tile, [num_examples, self.dim_model])
        
        ret = decoded_word_base + shared_lang_aux
        return ret 
    
    def get_feed(self, X, Y, src_lang, dest_lang, Z= None):        
        feed_dict = {self.source_words: X, self.target_words: Y}
        if (Z is not None):
            feed_dict[self.max_margin_neighbors] = Z
        return feed_dict

#     def get_prediction(self, nearest_ne, lang, pred, target, k_neighbors):
#         def dist(point, cur_point):
#             return np.linalg.norm(point-cur_point)

#         def f(row, cur_point):
#             dist_list = np.apply_along_axis(dist, 1, row, cur_point)
#             indices = np.argsort(dist_list)
#             return row[indices[len(indices)//2:]]
        
#         nearest_neighbors_pred = nearest_ne.knn(pred, lang, k_neighbors)
#         farthest_neighbors_target =[]
#         for cur_point,row in zip(target, nearest_neighbors_pred):
#             farthest_neighbors_target.append(f(row, cur_point))
        
#         return farthest_neighbors_target
        
    #train -> train[en_pt]
    def train(self, train, train_lang_pairs, batch_size, validation, test, test_lang_pairs,nearest_ne, nearest_ne_train ,\
              num_epochs=10, sess=None, log_file="./log_file.txt"):
        if sess == None:
            sess = tf.Session(graph = self.graph)
            sess.run(self.init)
                            
        with sess.as_default():
            assert sess is tf.get_default_session()
            
            #max data for a language pair/batch size
        max_data_size = int(max([len(x) for x in train])/batch_size)
        batch_counter_outer = 0
        try:
            with open(log_file, 'w') as log :
                log.write("Batch \t lang_pair \t loss\n")

                for epoch in range(num_epochs):
                    batch_index = np.zeros(shape=(len(train)), dtype=int)
                    for batch_number in range(max_data_size):
                        #make batches
                        batch_counter_outer += 1

                        for lang_pair_data_enum in enumerate(train):
                            log_string = ""
                            i, lang_pair_data = lang_pair_data_enum
                            lang_pair_data = np.array(lang_pair_data)

                            #rounding on the language pairs with less words in dictionary
                            if batch_index[i] >= len(lang_pair_data):
                                np.random.shuffle(train[i])
                                batch_index[i] = 0

                            cur_batch = lang_pair_data[batch_index[i]:(batch_index[i]+batch_size),:,:]
                            batch_index[i] += batch_size

                            X = cur_batch[:,0,:]
                            Y = cur_batch[:,1,:]

                            lang = train_lang_pairs[i].split('-')

                            A = self.all_lang_rep_enc[lang[0]]
                            B = self.all_lang_rep_dec[lang[1]]
                            #init = np.append(A,B)

                            # making shape to (1, 1* dim_lang) from (, 1*dim_lang )
                            A = A.reshape(1, np.shape(A)[0])
                            B = B.reshape(1, np.shape(B)[0])

                            #assign the lang representation for a run
                            sess_lang_rep_enc, sess_lang_rep_dec = sess.run([self.assign_lang_rep_enc_op, self.assign_lang_rep_enc_op] \
                                                          , feed_dict={self.lang_rep_enc_placeholder : A, self.lang_rep_dec_placeholder : B})

                            # get the negative examples wrt the best heuristic (near to prediction and far from target)
                            target_pred = sess.run([self.target_pred], self.get_feed(X, Y, lang[0], lang[1]))
                            Z = self.get_prediction(nearest_ne_train, lang[1], target_pred, Y, 80)

                            # Find k random negative samples for each data word in Y
                            #Z = lang_pair_data[:,1,:][random.sample(range(len(lang_pair_data)),k)]

                            #batch index more that lang pair data length not handled :P
                            _, train_loss, lang_rep_enc, lang_rep_dec, encoder, decoder= \
                                        sess.run([self.train_step, self.loss, self.lang_rep_enc, self.lang_rep_dec, self.encoder, self.decoder],\
                                                 self.get_feed(X, Y, lang[0], lang[1], Z))

                            log_string += str(batch_counter_outer) + " \t " + train_lang_pairs[i] + " \t " + str(train_loss/batch_size) + "\n"
                            log.write(log_string)

                            self.all_lang_rep_enc[lang[0]], self.all_lang_rep_dec[lang[1]] = lang_rep_enc[0], lang_rep_dec[0]

                            #test_code after every iteration
                            if (batch_counter_outer % 20 == 0):
                                self.test(test, test_lang_pairs, nearest_ne, sess)
                            #
                            #---------------------------------------------------------
                            #print("encoder : ", encoder)
                            #print("decoder : ", decoder)
                            #print("lang_encoder : ", lang_encoder)
                            #print("lang rep : ", lang_rep)
                            #print ("error : ", train_loss)
                            #if (lang[0] == 'pt'):
                            #    print("previous lang rep: ", A)
                            #    print("new lang rep :", lang_rep[0][:self.dim_lang])

                            #print (lang[0], lang_rep[0][:5])
                            #print (lang[1], lang_rep[0][5:])
                            print("Batch:" + str(batch_counter_outer))
                            #-----------------------------------------------------------
                    #save epoch
                    if epoch and epoch%10==0:
                        self.saver.save(sess, self.ckpt_path+self.model_name+".ckpt", global_step=epoch)

            #print losses #todo: format for train data with lang
            self.saver.save(sess, self.ckpt_path+self.model_name+".ckpt", global_step=num_epochs+1)
                #np.savetxt()
        except KeyboardInterrupt:
            print ('Interupted by keyboard')
            self.saver.save(sess, self.ckpt_path+self.model_name+".ckpt", global_step=epoch)

    def get_prediction(self, nearest_ne, lang, pred, target, k_neighbors):        
        #print(np.shape(pred))
        nearest_neighbors_pred = nearest_ne.knn(pred[0], lang, k_neighbors)
        
        farthest_neighbors_target = Parallel(n_jobs=1)(delayed(farthest_points)(row, cur_point) for cur_point,row in zip(target, nearest_neighbors_pred))
                
        return np.array(farthest_neighbors_target)
    
    
    def restore_last_session(self):
        saver = tf.train.Saver()
        # create a session
        sess = tf.Session()
        # get checkpoint state
        ckpt = tf.train.get_checkpoint_state(self.ckpt_path)
        # restore session
        if ckpt and ckpt.model_checkpoint_path:
            saver.restore(sess, ckpt.model_checkpoint_path)
        # return to user
        return sess

    def test(self, test, test_lang_pairs, nearest_neighbor, sess, log_file="./test_log_file.txt"):
        for lang_pair_data_enum in enumerate(test):
            i, lang_pair_data = lang_pair_data_enum
            lang_pair_data = np.array(lang_pair_data)
            
            lang = test_lang_pairs[i].split('-')
            A = self.all_lang_rep_enc[lang[0]]
            B = self.all_lang_rep_dec[lang[1]]
            
                        
            # making shape to (1, 1* dim_lang) from (, 1*dim_lang )
            A = A.reshape(1, np.shape(A)[0])
            B = B.reshape(1, np.shape(B)[0])

            #assign the lang representation for a run
            #sess_lang_rep = sess.run(self.assign_lang_rep_op, feed_dict={self.lang_rep_placeholder : init})
            sess_lang_rep_enc, sess_lang_rep_dec = sess.run([self.assign_lang_rep_enc_op, self.assign_lang_rep_enc_op] \
                                                          , feed_dict={self.lang_rep_enc_placeholder : A, self.lang_rep_dec_placeholder : B})

            
            X = lang_pair_data[:,0,:]
            Y = lang_pair_data[:,1,:]
            
            # Find k random negative samples for each data word in Y
            #todo: change this to nearest neighbor hueristic
            Z = lang_pair_data[:,1,:][random.sample(range(len(lang_pair_data)), self.k)]
            Z = np.tile(Z, (len(Y),1,1))
            #print (np.shape(Z))
            #print (Z)
            #batch index more that lang pair data length not handled :P
            test_loss, target_pred = sess.run([self.loss, self.target_pred], self.get_feed(X, Y, lang[0], lang[1], Z))
            near_points = nearest_neighbor.knn(target_pred, lang[1], 10)
            self.calculate_precision(near_points, Y)
            print('--------------------')
            print(lang)
            print("Test loss: ", test_loss)
            
            '''print('P@1: ', nn(1))
            print('P@5: ', nn(5))
            print('P@10: ', nn(10))'''
            break
     
    def calculate_precision(self, predictions_nn, actual):
        pos = 0
        total = len(actual)
        #print("actual:", actual[0])
        #print("pred:", predictions_nn[0][0])            
        pos = np.sum(Parallel(n_jobs=1)(delayed(match)(nn_list, y) for nn_list, y in zip(predictions_nn, actual)))
        '''
        for (i, y) in enumerate(actual):
            if y in predictions_nn[i]:
                pos+=1
        '''
        print(str(pos) + "out of " + str(total))

In [9]:
mlpm = model(ckpt_path="./ckpt/",lr=0.001)

start building graph
graph built


In [10]:
#mlpm.train(train_data_normalized,train_lang_pairs= ["pt-es", "pt-en", "en-es"], batch_size=40, validation=[], test= test_data_normalized, test_lang_pairs=["pt-es"])
mlpm.train(train_data,train_lang_pairs= ["pt-es", "pt-en", "en-es"], batch_size=40, validation=[], test= test_data, test_lang_pairs=["pt-es"], nearest_ne=nearest_ne, nearest_ne_train=nearest_ne_train)

Batch:1
Batch:1
Batch:1
Batch:2
Batch:2
Batch:2
Batch:3
Batch:3
Batch:3
Batch:4
Batch:4
Batch:4
Batch:5
Batch:5
Batch:5
Batch:6
Batch:6
Batch:6
Batch:7
Batch:7
Batch:7
Batch:8
Batch:8
Batch:8
Batch:9
Batch:9
Batch:9
Batch:10
Batch:10
Batch:10
Batch:11
Batch:11
Batch:11
Batch:12
Batch:12
Batch:12
Batch:13
Batch:13
Batch:13
Batch:14
Batch:14
Batch:14
Batch:15
Batch:15
Batch:15
Batch:16
Batch:16
Batch:16
Batch:17
Batch:17
Batch:17
Batch:18
Batch:18
Batch:18
Batch:19
Batch:19
Batch:19
6out of 86
--------------------
['pt', 'es']
Test loss:  1878.78
Batch:20
6out of 86
--------------------
['pt', 'es']
Test loss:  1862.71
Batch:20
5out of 86
--------------------
['pt', 'es']
Test loss:  1906.94
Batch:20
Batch:21
Batch:21
Batch:21
Batch:22
Batch:22
Batch:22
Batch:23
Batch:23
Batch:23
Batch:24
Batch:24
Batch:24
Batch:25
Batch:25
Batch:25
Batch:26
Batch:26
Batch:26
Batch:27
Batch:27
Batch:27
Batch:28
Batch:28
Batch:28
Batch:29
Batch:29
Batch:29
Batch:30
Batch:30
Batch:30
Batch:31
Batch:31
Batc